In [0]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [0]:
!pip install bert-tensorflow

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'OUTPUT_DIR_NAME'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: OUTPUT_DIR_NAME *****


#Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [0]:
from tensorflow import keras
import os
import re

In [0]:
trn = pd.read_json("https://github.com/ivantor0/realec-autoannotation/blob/master/gh_train.json?raw=true").reset_index(drop=True)
tst = pd.read_json("https://github.com/ivantor0/realec-autoannotation/blob/master/gh_test.json?raw=true").reset_index(drop=True)

In [0]:
trn

,entry,is_error,substring
0,"Some people believe that social networks, incl...",0,way
1,"The table shows underground of London, Paris, ...",0,in
2,The chart below shows changes in the number of...,0,of
3,It is argued that the mood of the employees ha...,0,all
4,﻿ Nowadays rates of crime by young people are ...,0,way
5,It can be clearly seen from the chart that the...,0,of
6,In USA's graph we see that number of eBook mar...,0,is
7,That ’ s why if people want to be not only hea...,0,disiases
8,"The majority of people ( 50,1%) was children i...",0,steadily
9,Dear Ms Lawrence I am writing to inform you to...,0,our


To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [0]:
train = trn.sample(10000)
test = tst.sample(10000)

In [0]:
train

,entry,is_error,substring
6488,"In my opinion, the government should allow pir...",1,large
7638,This paper is designed as follows. In the firs...,1,then
5639,And what to do if you can ’ t get your money b...,0,Allfilms
7044,Nowadays [MASK] competition between companies ...,0,the
12723,It reached its peak in 2012 when exceeded 100 ...,0,Until
7973,2013 year is a key one. Apple ’ s profits fell...,0,last
7764,The graph grows from 2000 to 2060. That means ...,0,to
2287,"In other words, this family provokes the risin...",0,Carve
10190,"Moreover, we have briliant example, when in wa...",0,studing
7972,The chart also illustrates the gender and regi...,0,data


In [0]:
train.columns

Index(['entry', 'is_error', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [0]:
DATA_COLUMN = 'entry'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
train_InputExamples

6488     <bert.run_classifier.InputExample object at 0x...
7638     <bert.run_classifier.InputExample object at 0x...
5639     <bert.run_classifier.InputExample object at 0x...
7044     <bert.run_classifier.InputExample object at 0x...
12723    <bert.run_classifier.InputExample object at 0x...
7973     <bert.run_classifier.InputExample object at 0x...
7764     <bert.run_classifier.InputExample object at 0x...
2287     <bert.run_classifier.InputExample object at 0x...
10190    <bert.run_classifier.InputExample object at 0x...
7972     <bert.run_classifier.InputExample object at 0x...
1681     <bert.run_classifier.InputExample object at 0x...
12074    <bert.run_classifier.InputExample object at 0x...
3234     <bert.run_classifier.InputExample object at 0x...
15763    <bert.run_classifier.InputExample object at 0x...
700      <bert.run_classifier.InputExample object at 0x...
3330     <bert.run_classifier.InputExample object at 0x...
13924    <bert.run_classifier.InputExample object at 0x.

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'mask' and split_tokens[index + 2] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [0]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 10000
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] in my opinion , the government should allow pirate files in internet as there are many people who cannot buy the original discs and use internet to listen to music or to watch film . on the other hand , musicians can make money on concerts and producers can have money from selling merchandise . despite the fact that there are a large number of pirate copies , i think that it is not a [MASK] problem , because a lot of people still go to cinema and buy cd discs . for example , the sites like kin ##op ##ois ##k . ru show that a lot of people prefer to go to the cinema instead of watching film at home . so [SEP] large [SEP]
INFO:tensorflow:input_ids: 101 1999 2026 5448 1010 1996 2231 2323 3499 11304 6764 1999 4274 2004 2045 2024 2116 2111 2040 3685 4965 1996 2434 15303 1998 2224 4274 2000 4952 2000 2189 2030 2000 3422 2143 1012 2006 1996 2060 2192 1010 5389 20

#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8eb9e43f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use tf.cast instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into OUTPUT_DIR_NAME/model.ckpt.
INFO:tensorflow:loss = 0.63074744, step = 0
INFO:tensorflow:global_step/sec: 0.563682
INFO:tensorflow:loss = 0.22760205, step = 101 (177.406 sec)
INFO:tensorflow:global_step/sec: 0.606754
INFO:tensorflow:loss = 0.18947995, step = 200 (164.812 sec)
INFO:tensorflow:global_step/sec: 0.607164
INFO:tensorflow:loss = 0.36446238, step = 300 (164.700 sec)
INFO:tensorflow:global_step/sec: 0.606738
INFO:tensorflow:loss = 0.2703094, step = 400 (1

Now let's use our test data to see how well our model did:

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-20T22:24:15Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from OUTPUT_DIR_NAME/model.ckpt-1874
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-20-22:27:05
INFO:tensorflow:Saving dict for global step 1874: auc = 0.59808373, eval_accuracy = 0.931, f1_score = 0.27368414, false_negatives = 459.0, false_positives = 231.0, global_step = 1874, loss = 0.39963886, precision = 0.3601108, recall = 0.22071308, true_negatives = 9180.0, true_positives = 130.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1874: OUTPUT_DIR_NAME/model.ckpt-1874


{'auc': 0.59808373,
 'eval_accuracy': 0.931,
 'f1_score': 0.27368414,
 'false_negatives': 459.0,
 'false_positives': 231.0,
 'global_step': 1874,
 'loss': 0.39963886,
 'precision': 0.3601108,
 'recall': 0.22071308,
 'true_negatives': 9180.0,
 'true_positives': 130.0}

Now let's write code to make predictions on new sentences:

In [0]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = [['In 2000 there war 11,1 milions boys and 21,6 milions girls, which is nearly similar to information about Africa in 2012. Next 12 years the quantity of boys dropped to 5,1 millions and girls – to 4,8 millions. The rest of world has changed its [MASK] of children without access to education twice, talking about girls, and less the twice thinking about boys. To sum up, from 2000 to 2012 there was trend of decreasing millions of boys and girls in all part of the world. The most successful changes are shown in South Asia.',
  'amount',
  1],
 ['Before thinking about how to solve the problem we should understand the reasons why it did appear. As it was stateted above, the reason of the increase in crime rates by young people is the exsessive control of them from parents and teachers. [MASK] latter in order to provide education, right behavior, and safety restrict a lot of things: from watching cruel TV-shows to visiting some places and communication with their friends. The reaction of young people is the protest against such norms. To show that they do not agree with the way parents and teachers bring them up, they do something prohibited.',
  'The',
  0],
 ['We are living in the liberty and democracy country, so not only artists but also everyone has truly rights to do what he or she wants. Creative artists, in particular, can feel respected when they have freedom to show their ideas to the world. They may feel [MASK] believe I them and always wait to see their new ideas such as words, pictures, music or film. In addition, this feeling can give them motivation which makes them create more wonderful production. Nevertheless, it will be too risk for the government to have no restriction on what artists do.',
  'people',
  0],
 ['It will be equal to less than 30%. In Japan number of elder calmly increases - there is no fast growth in 2030 like in the USA. In Sweden, growth [MASK] increases - the population of old people in 2040 will be near 25%. So, in all these countries the growth goes up and equal to 25%. Nowadays, the eldest country is Sweden.',
  'fluently',
  1]]

In [0]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 
INFO:tensorflow:tokens: [CLS] in 2000 there war 11 , 1 mil ##ions boys and 21 , 6 mil ##ions girls , which is nearly similar to information about africa in 2012 . next 12 years the quantity of boys dropped to 5 , 1 millions and girls – to 4 , 8 millions . the rest of world has changed its [MASK] of children without access to education twice , talking about girls , and less the twice thinking about boys . to sum up , from 2000 to 2012 there was trend of decreasing millions of boys and girls in all part of the world . the most successful changes are shown in south asia . [SEP] amount [SEP]
INFO:tensorflow:input_ids: 101 1999 2456 2045 2162 2340 1010 1015 23689 8496 3337 1998 2538 1010 1020 23689 8496 3057 1010 2029 2003 3053 2714 2000 2592 2055 3088 1999 2262 1012 2279 2260 2086 1996 11712 1997 3337 3333 2000 1019 1010 1015 8817 1998 3057 1516 2000 1018 1010 1022 8817 1012 1996 2717 1997 2088 20

Voila! We have a sentiment classifier!

In [0]:
predictions

[(['In 2000 there war 11,1 milions boys and 21,6 milions girls, which is nearly similar to information about Africa in 2012. Next 12 years the quantity of boys dropped to 5,1 millions and girls – to 4,8 millions. The rest of world has changed its [MASK] of children without access to education twice, talking about girls, and less the twice thinking about boys. To sum up, from 2000 to 2012 there was trend of decreasing millions of boys and girls in all part of the world. The most successful changes are shown in South Asia.',
   'amount',
   1],
  array([-1.7102101 , -0.19946091], dtype=float32),
  'Is an error'),
 (['Before thinking about how to solve the problem we should understand the reasons why it did appear. As it was stateted above, the reason of the increase in crime rates by young people is the exsessive control of them from parents and teachers. [MASK] latter in order to provide education, right behavior, and safety restrict a lot of things: from watching cruel TV-shows to visi

#Extracting the trained model
Using the following procedures we can extract the resulting state of now trained model.


In [0]:
os.listdir('.')

['.config',
 'OUTPUT_DIR_NAME',
 'wsites.json',
 'wsites_bert15x1-take1.json',
 'adc.json',
 'wsites_bert1x1-take1-750steps.json',
 'testigo.html',
 'sample_data']

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
os.chdir('OUTPUT_DIR_NAME')

In [0]:
os.listdir('.')

['model.ckpt-500.index',
 'model.ckpt-500.meta',
 'eval',
 'model.ckpt-1000.meta',
 'model.ckpt-1500.meta',
 'checkpoint',
 'model.ckpt-1500.index',
 'model.ckpt-1874.data-00000-of-00001',
 'events.out.tfevents.1550698238.909fd5543523',
 'model.ckpt-500.data-00000-of-00001',
 'model.ckpt-1874.index',
 'model.ckpt-1000.index',
 'model.ckpt-0.data-00000-of-00001',
 'graph.pbtxt',
 'model.ckpt-1000.data-00000-of-00001',
 'model.ckpt-0.index',
 'model.ckpt-1874.meta',
 'model.ckpt-0.meta',
 'model.ckpt-1500.data-00000-of-00001']

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'model.ckpt-1874.index'})
uploaded.SetContentFile('model.ckpt-1874.index')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1xlwANS_Kc3hwdz-_56eMFcIGlCOkZ02Q


In [0]:
os.chdir('..')

In [0]:
os.listdir('.')

['.config',
 'OUTPUT_DIR_NAME',
 'wsites.json',
 'wsites_bert15x1-take1.json',
 'adc.json',
 'wsites_bert1x1-take1-750steps.json',
 'testigo.html',
 'sample_data']

# Preparing existing model for REALEC production

In [0]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set)
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="blue">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [0]:
Model_name = 'bert15x1-take2-1874steps'  #@param {type:"string"}

In [0]:
import json

curname = "wsites_" + Model_name + ".json"

test_texts = {'DTi_50_2': """It is widely known that music labels and film makers lose a great deal of money every year from illegal copying and free internet sharing. Some people say that people who do that should be punished, some think that such men are the new «Robin Hoods». Let us take a look at this problem.
On the one hand, illegal copying is prohibited in almost all civilized countries and there is a reason for that. Music, books and films are considered as an intellectual property and it is quite uderstandable because artists are getting paid for composing, painting and film making only if their products sell, and if they do not have enough money for their living and creating they will just get another job, and this is why the law of intellectual property exists. It helps artists to get money they deserve and to have enough funds to make a quality product.
On the other hand, nowadays it is not so simple as it may look like at first. Musicians do not often need labels to record their masterpieces anymore because personal computers went so far that now you can record some high-quality sound right in your living room so you do not need to rent a studio for that. As for film makers, they get millions just by dressing the main character in a big logo T-shirt of some reach corporation. In fact, money, that a film company is paid for commercial, can sometimes fully cover the film making expences. 
To conclude, I would like to say that we should obey the law of intellectual property. If we want to live in a respectful society, but sometimes, I think we should also ask ourselves what we are paying for.""",
'VSa_105_1': """The graph provides information about development of the book sales system in 4 different countries (USA, Germany, China, UK) in 2014 and predicts future perspectives of this industry in 2018.
Generally speaking, it is observed a great increase of eBook on the market in the USA in 2018, which will surpass printed sources. By contrast, in other countries, except the UK, printed literature will remain constant position.
Despite the fact that print dominated on the book market of the USA in 2014 and exceeded eBook by a half (10,5 to 5,5 respectively), in 2018 it is forecasted that benefit of print literature will shrink on 3 billion US dollars. At the same time the income of eBook will grow on 3 billion US dollars and will be the most selling source of literature (8,5 billion $ of Ebook as against 7,5 billion $ of print).
In other countries the book market is not so developed as it in the USA. However, it is forecasted that in 2018 the ebook sales will slightly overtake printed book sales (2,3 to 2 respectively).
In Germany and China printed books will be the most popular kind of literature (6 to 4,2 respectively), especially in Germany print indicators remain stable.
But it also be a small growth in sales of Ebooks in these countries (from 1 to 1,5 in German and from 0,5 to 1 in China).""",
'NMya_90_1': """On the first graph we can see information about maximal and minimal average temperatures in Yakutsk. Graphs for both, the minimum and maximum have a parabolic appearance: they both have a steady increase until the hottest month of the year (july) and after that they both have as constant decrease down to december. Analazing this whole graph, you can point out several things. First of all, the difference of the highest and lowest temperatures between the coldest month and the hottest month is sixty degrees and fifty degrees respectively. The biggest difference between maximum and minimum temperatures is seen on march and it is about seventeen degrees. According to this graph, july is always the hottest month and january is the coldest.

On the second graph we can see similar temperature comparsions for Rio de Janeiro. Two lines represent minimum and maximum average temperatures for every month of the year. According to this graph, temperatures in Rio don’t change much throughout the year. the maximal difference of minimum and maximum temperature is on january, july and august and is approximately only seven degrees. For this city the coldest months are june and july, and the hottest are january and february.

Comparing the two graphs we notice this: average year temperature in rio is almost constant, comparing to Yakutsk and is never less then 18 °C, and the coldest and the hottest months in Rio are the hottest and the coldest in Yakutsk.
""",
'NChe_16_2': """In reacent time a lot of people claims that modern technology is a curse that leads to a health problems. It is no use to argue with this statement.
For begining,  many doctors already said, that such things like computers can caus a lot of prolems with heath. So, one of the most famous illness is damaged eyes. Then kids or adults spend a load of time next to monitor they starting to lose eyes sharphes. What is more, some scientiest claiws that some deuices cau lead euen cancer. There is no uniqe poiut betwine doctors, any way there is such kind of danger.
As for my, I find this problems not so unreducable. In general, all health desiases caused by modern techuologies were caused by ouer-use of this technologies and useing it in anpropriate way. So, if people start to control time that they spend with there deuices it will help to recduce a lot of problems with eyes. As a next step, we should understand that lightning aroun us is also very important. That means, that modern technologies haue some bad influens on people, but we can also dicrease that bad effect.
To sum up, wiede use of modern technology shown us that there are not only benefits in deuelopmeut of deuices, but also the great danger. But corect use of it can reduce to the minimum all harmful effects""",
'ESha_3_2': """Nowadays it is becoming cosier to express yourself by different ways. Some people do it by using words, some by using pictures of films. But is it clear to allow artist to do and to act how they want? 


There are so many ways to express your thoghts and feelings. People from ancient times show their ideas and thoughts by paintings and music. A lot of paintings of famous authors are held now in different galleries and big amount of people see them every day. But in long time ago just really talanted people become famous and well known artists.


Now situation is different. Every person can become an artist. Sometimes, people don't think how their ideas and works would influence other people's minds. Too many untalanted and unproffesional people create music, movies and paintings. It is really hard for a good artist to show himself in such amount of untalanted people.


In my opinion, government should allowed to express ideas and thoughts stronly really good artists. I their ideas are clear and they have something to show and tell people, they should do it. But if artist's works don't have any idea or logic or it can't bring anything good to public, there is nothing to show.


Every person have a chance to show itself by any way he choose. But at first, we should think, if he really wants it and if she really have something to say and show to bublicity. Only really good works should be shown to people, because it can influence them and their minds a lot.
"""}

outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 290
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 
INFO:tensorflow:tokens: [CLS] [MASK] is widely known that music labels and film makers lose a great deal of money every year from illegal copying and free internet sharing . some people say that people who do that should be punished , some think that such men are the new « robin hood ##s » . let us take a look at this problem . [SEP] it [SEP]
INFO:tensorflow:input_ids: 101 103 2003 4235 2124 2008 2189 10873 1998 2143 11153 4558 1037 2307 3066 1997 2769 2296 2095 2013 6206 24731 1998 2489 4274 6631 1012 2070 2111 2360 2008 2111 2040 2079 2008 2323 2022 14248 1010 2070 2228 2008 2107 2273 2024 1996 2047 1077 5863 7415 2015 1090 1012 2292 2149 2202 1037 2298 2012 2023 3291 1012 102 2009 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 16_pmUKOHS5lKXM2o-1QyVcPQohalCRZd
